[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the EECS JupyterHub as well as in the binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [444]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
#import datetime
#from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from uk_covid19 import Cov19API


In [445]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [446]:
 pip install uk-covid19


Note: you may need to restart the kernel to use updated packages.


In [447]:
England_filters = [
    'areaType=nation',
    'areaName=England'
]

England_structure = {
    "date": "date",
    "cases": "newCasesBySpecimenDate",
    "new_first_episodes": "newFirstEpisodesBySpecimenDate",
    "new_reinfection": "newReinfectionsBySpecimenDate"    
}

graph = Cov19API(filters = England_filters, structure = England_structure)

In [448]:
analysis = graph.get_json()
with open("graph.json", "wt") as file:
    json.dump(analysis, file)

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [449]:
def wrangle_data(rawdata, columns_name):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    with open(rawdata, "rt") as INFILE:
        input_data = json.load(INFILE)
    
    global datalist 
    datalist = input_data['data']
    
    date_index() 
    global x_index
    x_index = pd.date_range(startdate, enddate) 
    
    global dataframe
    dataframe = pd.DataFrame(index = x_index, columns = columns_name)
    
    for i in datalist: 
        date = parse_date(i['date'])
        for x in columns_name:
            if pd.isna(dataframe.loc[date, x]): 
                value = float(i[x]) if i[x]!=None else 0.0
                dataframe.loc[date, x] = value    
                
    dataframe.fillna(0.0, inplace=True)

    return dataframe

def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def date_index():      
    dates = [i['date'] for i in datalist]
    dates.sort() 
    global startdate
    global enddate
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])

In [450]:
graph_df = wrangle_data("graph.json", ['cases', 'new_first_episodes', 'new_reinfection']) 
graph_df

,cases,new_first_episodes,new_reinfection
2020-01-30,1.0,1.0,0.0
2020-01-31,0.0,0.0,0.0
2020-02-01,0.0,0.0,0.0
2020-02-02,1.0,1.0,0.0
2020-02-03,18.0,18.0,0.0
...,...,...,...
2022-11-19,2140.0,1424.0,716.0
2022-11-20,2476.0,1548.0,928.0
2022-11-21,3285.0,1987.0,1298.0
2022-11-22,2948.0,1798.0,1150.0


## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [451]:
def access_api():
    return {} 

In [452]:
def api_button_callback(button):
    apidata=access_api() 
    
    global dataframe
    dataframe = wrangle_data("graph.json", ['cases', 'new_first_episodes', 'new_reinfection'])

    refresh_graph()
    apibutton.icon = "check"
    apibutton.disabled = True 
    

In [453]:
apibutton = wdg.Button(
    description = 'Refresh',
    disabled = False,
    button_style = 'info', 
    icon = 'download'
)

apibutton.on_click(api_button_callback)


## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [454]:
def refresh_graph(): 
    current = scale.value
    if current == scale.options[0]:
        other = scale.options[1]
    else:
        other = scale.options[0]
    scale.value=other 
    scale.value=current    

def plot_graph(gscale, gcols):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
        
    ncols = len(gcols)

    if ncols > 0:
        dataframe[list(gcols)].plot(logy=logscale, ylabel = 'number of', xlabel = 'time')
        plt.show()   
    else:
        print("Click to select data for graph")
        print("CTRL-Click to select more than one category")

In [455]:
series = wdg.SelectMultiple(
    options = ['cases', 'new_first_episodes', 'new_reinfection'],
    value = ['cases', 'new_first_episodes', 'new_reinfection'],
    rows = 3,
    description = 'Number of:',
    disabled = False
)


scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    value = 'linear',
    description = 'Scale:',
    disabled = False)

control = wdg.HBox([series, scale])

    

## Graph of Cases, First Episode and New Infection of Covid in England
The following graph shows the number of cases, first episode and new infection of covid in England. 

1. Choose from the selection box for what statistics to show
2. Choose the scale for y-axis
3. Click the 'Refresh' box to get the latest data (optional) 

In [456]:
graph = wdg.interactive_output(plot_graph, {'gscale':scale, 'gcols': series})
display(control, apibutton, graph)
dataframe

Button(button_style='info', description='Refresh', icon='download', style=ButtonStyle())

Output()

,cases,new_first_episodes,new_reinfection
2020-01-30,1.0,1.0,0.0
2020-01-31,0.0,0.0,0.0
2020-02-01,0.0,0.0,0.0
2020-02-02,1.0,1.0,0.0
2020-02-03,18.0,18.0,0.0
...,...,...,...
2022-11-19,2140.0,1424.0,716.0
2022-11-20,2476.0,1548.0,928.0
2022-11-21,3285.0,1987.0,1298.0
2022-11-22,2948.0,1798.0,1150.0


## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*